In [ ]:
import os
import torch
import torch.nn.functional as F
from efficientnet_pytorch import EfficientNet
from torchvision import transforms
from PIL import Image
import pickle
import csv

IMG_DIR = './soundscape_melspec_images'
MODEL_PATH = 'efficientnet_birdclef.pth'
LABEL_PATH = 'class_to_label.pkl'
OUTPUT_CSV = 'soundscape_predictions.csv'

In [ ]:
with open(LABEL_PATH, 'rb') as f:
    label_dict = pickle.load(f)
idx_to_class = {v: k for k, v in label_dict.items()}

idx_to_class

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = EfficientNet.from_name('efficientnet-b0')
in_features = model._fc.in_features
model._fc = torch.nn.Linear(in_features, len(label_dict))
model.load_state_dict(torch.load(MODEL_PATH, map_location=device))
model = model.to(device)
model.eval()

In [ ]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

In [ ]:
with open(OUTPUT_CSV, 'w', newline='') as f:
    writer = csv.writer(f)
    writer.writerow(['filename'] + [idx_to_class[i] for i in range(len(label_dict))])
    
    for fname in sorted(os.listdir(IMG_DIR)):
        if not fname.endswith('.png'):
            continue
        try:
            path = os.path.join(IMG_DIR, fname)
            img = Image.open(path).convert('RGB')
            tensor = transform(img).unsqueeze(0).to(device)

            with torch.no_grad():
                output = model(tensor)
                probs = F.softmax(output, dim=1).cpu().numpy().flatten()

            # 소수점 17자리로 포맷팅
            probs_formatted = [f"{p:.17f}" for p in probs]
            writer.writerow([fname[:-4]] + probs_formatted)
        
        except Exception as e:
            print(f"❌ {fname} error: {e}")
